<a href="https://colab.research.google.com/github/brianr-v/Seminario-de-modelacion/blob/main/Monte_Carlo_opciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Calculo de prima de opciones por método de Monte Carlo**

Primero importamos las librerías numpy y yfinance. La primera nos servira para determinar números aleatorios con distribución normal estandar, la segunda librería nos servira para exportar los datos historicos del valor de una acción en particular, en este caso AAPL (Apple).

In [2]:
import numpy as np
import yfinance as yf

Despues creo una función que nombro "monte_carlo", que tiene parametros de entrada:
* $S_0$: Precio actual de la acción.
* $K$: Precio de ejercicio, es decir a lo que acordamos que lo vamos a comprar en un futuro.
* $r$: Tasa libre de riesgo.
* $\sigma$: La volatilidad de la acción.
* tipo de acción: Podemos elejir entre 'CALL' y 'PUT'.
* n: Número de simulaciones.

En general se utilizan dos formulas. Una enla que calculamos el precio aproximado de una acción, donde se utiliza el módelo Browniano estandar que es la siguiente formula.
$S_T=S_0*exp[r-\frac{\sigma^2}{2})*T + \sigma\in\sqrt(T)]$

Despues determinamos los rendimientos promdeios de una opción CALL y PUT:
* CALL: $Max[S_T - K,0]$ y  $exp(-r*T)*Max(S_T-K)$
* PUT: $Max[K-S_T,0]$ y  $ exp(-r*T)*Max(K-S_T)$

In [3]:
def monte_carlo(S0, K, r, sigma, T, tipo_opcion, num_simualcion):
    dt = T/252              # Establecmos que hay 252 dias habiles en un año
    pasos = int(T*252)      # Número de pasos de tiempo
    z = np.random.normal(0, 1, (num_simualcion, pasos)) # Numeros alatoriso con distribución normal
    # Simulacion de precios del activo subyacente
    S = S0*np.exp(np.cumsum( (r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z, axis=1) )

    if tipo_opcion == 'CALL':
        payoff = np.maximum(S[:, -1] - K, 0)
    elif tipo_opcion == 'PUT':
        payoff = np.maximum(K - S[:, -1], 0)

    # Calculo del valor presente del pago de la opción
    valor_payoff = np.exp(-r*T) * payoff

    # CAlculo del valor presente del pago de la opción
    valor_opcion = np.mean(valor_payoff)

    return valor_opcion

Despues solo determino los datos iniciales, calculo la volatilidad y llamo a la función "monte_carlo".

In [4]:
AAPL_Ticker = yf.Ticker("AAPL")                     # Crear un objeto Ticker para la acción Apple

precio_hoy = AAPL_Ticker.history(period='1d')['Close'][0]
S0 = precio_hoy
K = 155                                              # Precio de ejercicio de la opción
r = 0.0517                                          # Tasa de interes libre de riesgo

# Volatilidad del activo subyecente
precio_historico = AAPL_Ticker.history(period='1y')
rendimientos_diarios = precio_historico['Close'].pct_change()  # Calcula rendimientos diarios
sigma = rendimientos_diarios.std()                  # Calcula la volatilidad como la desviación estandar de los rendimientos
T = 1                                               # Tiempo hasta el vencimiento de la opción en años
simulaciones = 10000                                # Número de simulaciones

# Valuación de una acción
print("\nPrecio del activo hoy: ", precio_hoy)
print("Precio de ejercicio es: ", K)
opcion_CALL = monte_carlo(S0, K, r, sigma, T, 'CALL', simulaciones)
print("Precio de la opción CALL: ", opcion_CALL)

# Valuación de una acción PUT
opcion_PUT = monte_carlo(S0, K, r, sigma, T, 'PUT', simulaciones)
print("Precio de la opción PUT: ", opcion_PUT)



Precio del activo hoy:  166.89999389648438
Precio de ejercicio es:  155
Precio de la opción CALL:  19.708591444769674
Precio de la opción PUT:  0.0
